## Exercise: Heat diffusion on CPUs and GPU

**NOTE: This notebook should be run with multiple Julia threads!**

#### Exercise objective
1) Understand how to (step-by-step) transform a vectorized numerical code into a faster, multithreaded loop-variant and, eventually, into a CUDA kernel running on a GPU.
2) Compare the performance of different variants of the code (vectorized CPU, vectorized GPU, multithreaded loop CPU, CUDA kernel GPU).

#### The problem
We consider the heat equation, a partial differential equation (PDE) describing the diffusion of heat over time. The PDE reads

$$ \dfrac{\partial T}{\partial t} = \alpha \left( \dfrac{\partial^2 T}{\partial x^2} + \dfrac{\partial^2 T}{\partial y^2} \right), $$

where the temperature $T = T(x,y,t)$ is a function of space ($x,y$) and time ($t$) and $\alpha$ is a scaling coefficient. Specifically, we'll consider a simple two-dimensional square geometry. As the initial condition - the starting distribution of temperature across the geometry - we choose a ["Gaussian"](https://en.wikipedia.org/wiki/Gaussian_function#:~:text=In%20mathematics%2C%20a%20Gaussian%20function,characteristic%20symmetric%20%22bell%20curve%22%20shape) positioned in the center. In summary, the starting configuration looks like this:

![](../../../other/imgs/heat_diffusion_initial_condition.png)

#### Numerical approach
1) We discretize space (`dx`, `dy`) and time (`dt`) and evaluate everything on a grid.
2) We use the basic [finite difference method](https://en.wikipedia.org/wiki/Finite_difference_method) to compute derivatives on the grid, e.g.
$$
\dfrac{\partial T}{\partial x}(x_i) \approx \dfrac{f(x_{i+1}) - f(x_i)}{\Delta x} 
$$
3) We use a two-step process:
    a) Compute the first-order spatial derivates.
    b) Then, update the temperature field (time integration).
$$ 
\begin{align}
\partial x &= \dfrac{\Delta T}{\Delta x} \\
\partial y &= \dfrac{\Delta T}{\Delta y} \\
\Delta T &= \alpha\Delta t \left( \dfrac{\Delta (\partial x)}{\Delta x} + \dfrac{(\Delta \partial y)}{\Delta y} \right)
\end{align}
$$

#### Performance metric
Note that the derivatives give our numerical solver the character of a [stencil](https://en.wikipedia.org/wiki/Iterative_Stencil_Loops). Stencils are typically memory bound, that is, data transfer is dominating over FLOPs and consequently performance is limited by the rate at which memory is transferred between memory and the arithmetic units. For this reason we will measure the performance in terms of an [effective memory throughput metric](https://github.com/omlins/ParallelStencil.jl#performance-metric)

$$T_\textrm{eff} = \dfrac{2 n_x n_y \delta}{t_{\textrm{it}}} \cdot 10^{-9} \quad [\textrm{GB/s}]$$

where $t_{\textrm{it}}$ is the time per iteration and $\delta$ is the arithmetic precision (8 or 4 bytes for `Float64` or `Float32` respectively).

## CPU - Vectorized

Below you find a code snippet for the integration of the heat equation. 

**Task 1**

1) Implement the missing piece, that is, the update of the temperature field.

**Remarks**

* Finite difference method: You can use `diff(A, dims=1) ./ dx` to compute the partial derivative of a field `A` in the x-direction (`dims=2` corresponds to y-direction).
* Don't update the temperature `T` at the boundary of the 2D square geometry. That is, only update the inner part `T[2:(end - 1), 2:(end - 1)]`.
* When implemented correctly, you should get this animation:

![](../../../other/imgs/heat_diffusion_animation.gif)

In [ ]:
using Plots
using Printf

@views function heat_2D_animation()
    # physical parameters
    lx, ly = 10.0, 10.0 # spacial dimension
    α = 1.0 # coefficient
    
    # spatial grid
    nx, ny = 128, 128
    dx, dy = lx/nx, ly/ny
    xc = range(start=dx/2, stop=lx-dx/2, length=nx)
    yc = range(start=dy/2, stop=ly-dy/2, length=ny)

    # time discretization
    dt = min(dx^2,dy^2)/α/4.1
    nt = 400 # timestepts
    
    # initial condition - gaussian sitting at the center
    T = exp.(.-(xc.-lx./2.0).^2 .-(yc.-ly./2.0)'.^2)

    # preallocation
    ∂x = zeros(nx-1,ny-2)
    ∂y = zeros(nx-2,ny-1)
    
    # time loop
    @gif for it = 1:nt 
        # -------- stencil kernel --------
        # first order derivatives
        ∂x .= diff(T[:, 2:(end - 1)], dims = 1) ./ dx
        ∂y .= diff(T[2:(end - 1), :], dims = 2) ./ dy

        #
        # !!!! Task 1 TODO: update T
        #
        
        # --------------------------------

        # plotting
        heatmap(xc, yc, T', xlabel="x", ylabel="y", title="Heat Diffusion, i=$it", clims=(0.,1.))
    end every 5
end

heat_2D_animation()

### Performance (without animation)
Let's get rid of the animation and measure the performance of our implementation. Please copy your solution for Task 1 above and paste it at the marked position.

In [ ]:
using Printf

@views function heat_2D(; ngrid=1024)
    # physical parameters
    lx, ly = 10.0, 10.0 # spacial dimension
    α = 1.0 # coefficient

    # spatial grid
    nx, ny = ngrid, ngrid
    dx, dy = lx / nx, ly / ny
    xc = range(start = dx / 2, stop = lx - dx / 2, length = nx)
    yc = range(start = dy / 2, stop = ly - dy / 2, length = ny)

    # time discretization
    dt = min(dx^2, dy^2) / α / 4.1
    nt = 400 # timestepts

    # initial condition - gaussian sitting at the center
    T = exp.(.-(xc .- lx ./ 2.0) .^ 2 .- (yc .- ly ./ 2.0)' .^ 2)

    # preallocation
    ∂x = zeros(nx - 1, ny - 2)
    ∂y = zeros(nx - 2, ny - 1)

    # time loop
    t0 = Base.time()
    for it in 1:nt
        # time measurement (skip first 10 iterations)
        if it == 11
            t0 = Base.time()
        end

        # -------- stencil kernel --------
        # first order derivatives
        ∂x .= diff(T[:, 2:(end - 1)], dims = 1) ./ dx
        ∂y .= diff(T[2:(end - 1), :], dims = 2) ./ dy
        # update T
        
        #
        # !!!! TODO: update T (just copy your solution from Task 1 here as well)
        #

        # --------------------------------
    end
    time_s = Base.time() - t0
    T_eff = round((2 * nx * ny * sizeof(eltype(T)) / (time_s / (nt - 10))) * 1e-9, sigdigits = 2)
    @printf("T_eff = %1.2f GB/s, Time = %1.4e s \n", T_eff, time_s)
    return nothing
end

In [ ]:
println("CPU - Vectorized:")
heat_2D()

## CPU - Loop Multithreaded
To multithread our stencil computation - and to later transform it into a CUDA kernel - we need a version of our numerical solver in terms of loops rather than broadcasting (vectorized). Below you find a corresponding code snippet where we've outsourced the first-order derivation computation into a function `compute_first_order_loop_mt!` and the temperature field update into `update_T_loop_mt!`. Like above, the first-order derivative computation is already implemented. Check it out to understand what has changed.

**Task 2**

1) Implement the missing piece, that is, the update of the temperature field via a multithreaded loop.

**Remarks**

* To respect column-major order (memory layout), the y-loop should be the outer loop and the x-loop the inner one.
* Use `@threads :static for ...` to parallelize the outer y-loop.

In [ ]:
using ThreadPinning
using Printf

if Threads.nthreads() == 1
    error("This part is supposed to be run with multiple Julia threads.")
end
pinthreads(:numa)

function compute_first_order_loop_mt!(∂x, ∂y, T, dx, dy, nx, ny)
    Threads.@threads :static for iy in 2:(ny - 1)
        for ix in 1:(nx - 1)
            ∂x[ix, iy - 1] = (T[ix + 1, iy] - T[ix, iy]) / dx
        end
    end
    Threads.@threads :static for iy in 1:(ny - 1)
        for ix in 2:(nx - 1)
            ∂y[ix - 1, iy] = (T[ix, iy + 1] - T[ix, iy]) / dy
        end
    end
    return nothing
end

function update_T_loop_mt!(T, ∂x, ∂y, dt, α, dx, dy, nx, ny)
    
    #
    # Task 2 TODO: Implement update of T via multithreaded loops.
    #              See `compute_first_order_loop_mt!` for inspiration.
    #

    return nothing
end

@views function heat_2D_loop_multithreaded(; ngrid=1024)
    # physical parameters
    lx, ly = 10.0, 10.0 # spacial dimension
    α = 1.0 # coefficient

    # spatial grid
    nx, ny = ngrid, ngrid
    dx, dy = lx / nx, ly / ny
    xc = range(start = dx / 2, stop = lx - dx / 2, length = nx)
    yc = range(start = dy / 2, stop = ly - dy / 2, length = ny)

    # time discretization
    dt = min(dx^2, dy^2) / α / 4.1
    nt = 400 # timestepts

    # initial condition - gaussian sitting at the center
    T = exp.(.-(xc .- lx ./ 2.0) .^ 2 .- (yc .- ly ./ 2.0)' .^ 2)

    # preallocation
    ∂x = zeros(nx - 1, ny - 2)
    ∂y = zeros(nx - 2, ny - 1)

    # time loop
    t0 = Base.time()
    for it in 1:nt
        # time measurement (skip first 10 iterations)
        if it == 11
            t0 = Base.time()
        end

        # -------- stencil kernel --------
        # first order derivatives
        compute_first_order_loop_mt!(∂x, ∂y, T, dx, dy, nx, ny)
        # update T
        update_T_loop_mt!(T, ∂x, ∂y, dt, α, dx, dy, nx, ny)
        # --------------------------------
    end
    time_s = Base.time() - t0
    T_eff = round((2 * nx * ny * sizeof(eltype(T)) / (time_s / (nt - 10))) * 1e-9, sigdigits = 2)
    @printf("T_eff = %1.2f GB/s, Time = %1.4e s \n", T_eff, time_s)
    return nothing
end

In [ ]:
println("CPU - Loop Multithreaded:")
heat_2D_loop_multithreaded()

## GPU - Vectorized
The simplest way to make our numerical code run on the GPU is to use array abstractions, that is, `CuArray`s instead of the regular `Array`s. Below you find precisely the same code snippet as for Task 1 (CPU vectorized).

**Task 3**

1) Initialize the arrays `T`, `∂x`, and `∂y` on the GPU by making the `CuArray`s. Leave the rest of the code the same.
2) Do you notice a change in performance?

**Remarks**

* Since the initialization of the arrays isn't part of our time measurement you can either initialize the arrays directly on the GPU or move them there before the computation. Up to you :)
* **Important:** For a fair comparison in terms of `T_eff` (effective memory bandwidth), we need to use `CuArray{Float64}` rather than just `CuArray`, which defaults to `CuArray{Float32}`.

In [ ]:
using CUDA
using Printf

@views function heat_2D_gpu(; ngrid=1024)
    # physical parameters
    lx, ly = 10.0, 10.0 # spacial dimension
    α = 1.0 # coefficient

    # spatial grid
    nx, ny = ngrid, ngrid
    dx, dy = lx / nx, ly / ny
    xc = range(start = dx / 2, stop = lx - dx / 2, length = nx)
    yc = range(start = dy / 2, stop = ly - dy / 2, length = ny)

    # time discretization
    dt = min(dx^2, dy^2) / α / 4.1
    nt = 400 # timestepts

    # initial condition - gaussian sitting at the center
    #
    # Task 3 TODO: Put T on the GPU by making it a `CuArray`
    #
    T = exp.(.-(xc .- lx ./ 2.0) .^ 2 .- (yc .- ly ./ 2.0)' .^ 2)

    # preallocation
    #
    # Task 3 TODO: Put ∂x and ∂y on the GPU by making it a `CuArray`
    #
    ∂x = zeros(nx - 1, ny - 2)
    ∂y = zeros(nx - 2, ny - 1)

    # time loop
    t0 = Base.time()
    for it in 1:nt
        # time measurement (skip first 10 iterations)
        if it == 11
            t0 = Base.time()
        end

        # -------- stencil kernel --------
        # first order derivatives
        ∂x .= diff(T[:, 2:(end - 1)], dims = 1) ./ dx
        ∂y .= diff(T[2:(end - 1), :], dims = 2) ./ dy
        # update T
        T[2:(end - 1), 2:(end - 1)] .= T[2:(end - 1), 2:(end - 1)] .+
                                       dt .* α .* (diff(∂x, dims = 1) ./ dx .+
                                        diff(∂y, dims = 2) ./ dy)
        # --------------------------------
    end
    time_s = Base.time() - t0
    T_eff = round((2 * nx * ny * sizeof(eltype(T)) / (time_s / (nt - 10))) * 1e-9, sigdigits = 2)
    @printf("T_eff = %1.2f GB/s, Time = %1.4e s \n", T_eff, time_s)
    return
end

In [ ]:
println("GPU - Vectorized:")
heat_2D_gpu()

## GPU - CUDA Kernels
Finally, we want to transform our loop implementations of `compute_first_order_loop_mt!` and `update_T_mt!` above into CUDA kernels. To that end, we simply replace the (multithreaded) `for`-loops by `if`-conditions that make sure that the indices stay within the bounds of the arrays. When the kernel is later spawned with `@cuda`, different GPU-threads will run our kernel function with different indices and thus realize the full update.

**Task 4**

1) Implement the update of the temperature field as a CUDA kernel.
2) At the marked position, spawn the `update_T_gpu!` kernel with `@cuda` using `cublocks`-many blocks and `cuthreads`-many threads.

**Remarks**

* You may take `compute_first_order_gpu!` as inspiration.

In [ ]:
using CUDA
using Printf

function compute_first_order_gpu!(∂x, ∂y, T, dx, dy, nx, ny)
    ix = (blockIdx().x - 1) * blockDim().x + threadIdx().x # thread ID, dimension x
    iy = (blockIdx().y - 1) * blockDim().y + threadIdx().y # thread ID, dimension y

    if ix <= nx - 1
        if 2 <= iy <= ny - 1
            ∂x[ix, iy - 1] = (T[ix + 1, iy] - T[ix, iy]) / dx
        end
    end
    if 2 <= ix <= nx - 1
        if iy <= ny - 1
            ∂y[ix - 1, iy] = (T[ix, iy + 1] - T[ix, iy]) / dy
        end
    end
    return nothing
end

function update_T_gpu!(T, ∂x, ∂y, dt, α, dx, dy, nx, ny)
    
    #
    # Task 4 TODO: Implement the temperature update as a CUDA kernel.
    #              See `compute_first_order_gpu!` above for inspiration.
    #

    return nothing
end

@views function heat_2D_gpu_kernels(; ngrid=1024)
    # physical parameters
    lx, ly = 10.0, 10.0 # spacial dimension
    α = 1.0 # coefficient

    # spatial grid
    blockx, blocky = 32, 32
    gridx, gridy = ngrid ÷ 32, ngrid ÷ 32
    cuthreads = (blockx, blocky, 1)
    cublocks = (gridx, gridy, 1)
    nx, ny = blockx * gridx, blocky * gridy
    dx, dy = lx / nx, ly / ny
    xc = range(start = dx / 2, stop = lx - dx / 2, length = nx)
    yc = range(start = dy / 2, stop = ly - dy / 2, length = ny)

    # time discretization
    dt = min(dx^2, dy^2) / α / 4.1
    nt = 400 # timestepts

    # initial condition - gaussian sitting at the center
    T = CuArray{Float64}(exp.(.-(xc .- lx ./ 2.0) .^ 2 .- (yc .- ly ./ 2.0)' .^ 2))

    # preallocation
    ∂x = CUDA.zeros(Float64, nx - 1, ny - 2)
    ∂y = CUDA.zeros(Float64, nx - 2, ny - 1)

    # time loop
    t0 = Base.time()
    for it in 1:nt
        # time measurement (skip first 10 iterations)
        if it == 11
            t0 = Base.time()
        end

        # -------- stencil kernel --------
        # first order derivatives
        CUDA.@sync @cuda blocks=cublocks threads=cuthreads compute_first_order_gpu!(∂x, ∂y,
                                                                                    T, dx,
                                                                                    dy, nx,
                                                                                    ny)

        # update T
        
        #
        # Task 4 TODO: Spawn the `update_T_gpu!` kernel with `@cuda`.
        #
        
        # --------------------------------
    end
    time_s = Base.time() - t0
    T_eff = round((2 * nx * ny * sizeof(eltype(T)) / (time_s / (nt - 10))) * 1e-9, sigdigits = 2)
    @printf("T_eff = %1.2f GB/s, Time = %1.4e s \n", T_eff, time_s)
    return
end

In [ ]:
println("GPU - CUDA Kernels:")
heat_2D_gpu_kernels()

## Performance Comparison

In [ ]:
println("CPU - Vectorized:")
heat_2D()

println("CPU - Loop Multithreaded:")
heat_2D_loop_multithreaded()

println("GPU - Vectorized:")
heat_2D_gpu()

println("GPU - CUDA Kernels:")
heat_2D_gpu_kernels()